# Find list of bilinguals candidates

Here we find users to be contacted for the translation task. We combine two strategies:

* Users self reporting their languages skills (3 or more) in Babel, and editing at least 100 times in each languages.
* The top users using the translation tool. 

In [1]:
#setup langs

langs = ['fr','es','ru','ar','en','ja'] #languages included


## Babel

In [2]:
import pandas as pd
from itertools import combinations

babel = pd.read_csv('allwiki-babel-out-users.tsv.tar.gz',sep='\t',index_col=0)
babel['username'] = babel.index

In [3]:
levels = ['3','4','5','N']  #babel labels accepted
minEdits = 10 # olunteers need to have edited more than %minEdits times in both langs
usersBabel =[]
for lang1,lang2 in combinations(langs,2):
    babel[babel[lang1+'_proficiency'].isin(levels)]
    tmp = babel[babel[lang1+'_proficiency'].isin(levels)]
    tmp = tmp[tmp[lang2+'_proficiency'].isin(levels)]
    tmp  = tmp[tmp['%swiki_editcount' % lang1] > minEdits]
    tmp  = tmp[tmp['%swiki_editcount' % lang2] > minEdits]
    tmp['lang1'] = lang1
    tmp['lang2'] = lang2
    usersBabel.append(tmp[['username','lang1','lang2']][0:5])
    
outputBabel = pd.concat(usersBabel)

## Translation tool (Cx)

In [20]:
from sqlalchemy import create_engine    
from itertools import permutations

                                            
engine = create_engine('mysql://XXXXX@replicas/wikishared')
usersCx = []
for lang1,lang2 in permutations(langs,2):
    query = """
    SELECT trans.cnt as cnt ,global.gu_id,global.gu_name as username FROM (SELECT translation_started_by,  count(translation_target_title) as cnt FROM cx_translations WHERE (translation_status = 'published' OR translation_target_url IS NOT null) AND  translation_target_language = '%s' AND translation_source_language='%s' GROUP BY translation_started_by ORDER BY count(translation_target_title) DESC LIMIT 5) AS trans INNER JOIN centralauth.globaluser as global ON global.gu_id = trans.translation_started_by WHERE cnt >10;
    """ % (lang1,lang2)
    tmp = pd.read_sql_query(query,engine)
    tmp['lang1'] = lang1
    tmp['lang2'] = lang2
    tmp['username'] = tmp.username.str.decode("utf-8")
    tmp['Translations'] = tmp.cnt
    usersCx.append(tmp[['username','lang1','lang2','Translations']][0:5])
outputCx= pd.concat(usersCx)

## Save output

In [21]:
output = pd.concat([outputCx,outputBabel]).drop_duplicates(subset=['username'])

In [22]:
output.to_excel('peopleToContact.xls',index=False)

## Get stats

In [23]:
output['pairs'] =  output[['lang1', 'lang2']].apply(lambda x: ','.join(sorted(x)), axis=1)

In [24]:
output.pairs.value_counts()

es,fr    15
en,es    14
en,fr    13
en,ru    10
ar,fr     9
es,ru     6
en,ja     5
ar,en     4
fr,ru     3
fr,ja     2
ar,es     2
Name: pairs, dtype: int64

We get at least 6 candidates pair for each pair